In [1]:
from azure.storage.blob import BlobClient
import tarfile
import os

compute_data_folder = "../../data/"
zip_files = ["Segments_mat_gt_bb_S11.tgz", 
             "Segments_mat_gt_bb_S5.tgz", "Segments_mat_gt_bb_S6.tgz", "Segments_mat_gt_bb_S7.tgz",
            "Segments_mat_gt_bb_S8.tgz", "Segments_mat_gt_bb_S9.tgz"]
#zip_files = ["Segments_mat_gt_bb_S1.tgz"]
conn_str = "DefaultEndpointsProtocol=https;AccountName=nyposeestimation;AccountKey=Uf7Ket/F5XqlnHJx3vbJ/HDJNNzGpnUIuNo2rCg4CxAVHgF3+V574b2Siw6EM0Ef5+Rw5aWUd2k++AStXDZ8fQ==;EndpointSuffix=core.windows.net"
container_name = "human36m"
blob_data_folder = "data/"

In [2]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import torchvision.transforms.functional as F
from torchvision.io import read_video # extract frames
import os
import h5py
from torchvision.utils import save_image
import torchvision.transforms as T
import cv2

In [3]:
plt.rcParams["savefig.bbox"] = "tight"
# sphinx_gallery_thumbnail_number = 2

def plot(imgs, **imshow_kwargs):
    if not isinstance(imgs[0], list):
        # Make a 2d grid even if there's just 1 row
        imgs = [imgs]

    num_rows = len(imgs)
    num_cols = len(imgs[0])
    _, axs = plt.subplots(nrows=num_rows, ncols=num_cols, squeeze=False)
    for row_idx, row in enumerate(imgs):
        for col_idx, img in enumerate(row):
            ax = axs[row_idx, col_idx]
            img = F.to_pil_image(img.to("cpu"))
            ax.imshow(np.asarray(img), **imshow_kwargs)
            ax.set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])

    plt.tight_layout()
    plt.show()

# get frames from video path
def get_frames(video_path):
    frames, _, _ = read_video(video_path, output_format="TCHW", pts_unit="sec")
    return frames

# make frame suitable for inference
def make_frame_suitable(frame):
    transform = T.ToPILImage()
    frame = transform(frame)
    frame = frame.resize((256, 256)) # resize to 256 x 256
    return frame

def frame_resize_to_nparr(frame):
    transform = T.ToPILImage()
    frame = transform(frame)
    frame = frame.resize((256, 256)) # resize to 256 x 256
    return np.array(frame)

def frame_to_tensor(frame):
    transform = T.ToPILImage()
    frame = transform(frame)
    frame = frame.resize((256, 256)) # resize to 256 x 256
    toTensor = T.ToTensor()
    return toTensor(frame)

# get list of all subjects
def get_subjects():
    video_folders = os.listdir(compute_data_folder)[3:]
    return video_folders

def get_video_paths(subject):
    video_paths = os.listdir(compute_data_folder + subject + "/videos/")[2:]
    return video_paths

In [4]:
subjects = get_subjects()
#subjects = ['S1', 'S11', 'S5']

# for each subject
for subject in subjects:
    video_paths = get_video_paths(subject)
    path_prefix = compute_data_folder + subject
    
    save_prefix = compute_data_folder + "h36m/training/" + subject + "/frames/"
    
    print("Processing subject ", subject)
    
    # for each video (i.e. activity), get list of frames
    
    for j in range(len(video_paths)):
        #if j % 3 == 0:
        #    print("----video ", (j+1) / len(video_paths))
        
        video_path = path_prefix + "/videos/" + video_paths[j]
        activity = video_paths[j][:-4]
        
        save_prefix_vid = save_prefix + activity + "/"
        
        # get frames for this video
        vidcap = cv2.VideoCapture(video_path)
        i = 0
        while True:
            success,image = vidcap.read()
            if not success:
                break
            image = frame_resize_to_nparr(image)
            cv2.imwrite(save_prefix_vid + ("frame%04d.png" % i), image)     # save frame as JPEG file
            i += 1

Processing subject  S1


KeyboardInterrupt: 